In [0]:
# pip install pyspark

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, split

In [0]:
SparkSession

pyspark.sql.session.SparkSession

In [0]:
spark = SparkSession.builder.appName('Youtube_Data_Analysis').getOrCreate()

In [0]:
spark

In [0]:
spark.sparkContext

<SparkContext master=spark://10.139.64.5:7077 appName=Databricks Shell>

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

In [0]:
# Load the CSV file without headers
# Read the CSV file with space separator
df = spark.read.csv('file:/Workspace/Users/d-user11@heu.ai/youtubedata.csv',header=False,inferSchema=True)
df.toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22
0,QuRYeRnAuXM,EvilSquirrelPictures,1135.0,Pets & Animals,252.0,1075.0,4.96,46.0,86.0,gFa1YMEJFag,nRcovJn9xHg,3TYqkBJ9YRk,rSJ8QZWBegU,0TZqX5MbXMA,UEvVksP91kg,ZTopArY7Nbg,0RViGi2Rne8,HT_QlOJbDpg,YZev1imoxX8,8qQrrfUTmh0,zQ83d_D2MGs,u6_DQQjLsAw,73Wz9CQFDtE
1,3TYqkBJ9YRk,hggh22,1135.0,Comedy,169.0,228.0,5.00,5.0,3.0,QuRYeRnAuXM,gFa1YMEJFag,UEvVksP91kg,rSJ8QZWBegU,nRcovJn9xHg,sVkuOk4jmCo,ZTopArY7Nbg,HT_QlOJbDpg,0RViGi2Rne8,ShhClb6J-NA,g9e1alirMhc,YZev1imoxX8,I4yKEK9o8gA,zQ83d_D2MGs
2,rSJ8QZWBegU,TimeGem,1135.0,Entertainment,95.0,356.0,4.31,13.0,1.0,QuRYeRnAuXM,gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,nRcovJn9xHg,sVkuOk4jmCo,ZTopArY7Nbg,gBcu22Vv1nY,HT_QlOJbDpg,0RViGi2Rne8,ShhClb6J-NA,g9e1alirMhc,YZev1imoxX8,I4yKEK9o8gA
3,nRcovJn9xHg,wooochacha,1135.0,Entertainment,118.0,1115.0,2.23,57.0,73.0,QuRYeRnAuXM,gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,ZTopArY7Nbg,gBcu22Vv1nY,HT_QlOJbDpg,0RViGi2Rne8,ShhClb6J-NA,g9e1alirMhc,YZev1imoxX8,I4yKEK9o8gA,zQ83d_D2MGs,1GKaVzNDbuI
4,UEvVksP91kg,johnx113,1135.0,Entertainment,83.0,281.0,2.67,9.0,16.0,gFa1YMEJFag,QuRYeRnAuXM,3TYqkBJ9YRk,0TZqX5MbXMA,rSJ8QZWBegU,nRcovJn9xHg,sVkuOk4jmCo,ZTopArY7Nbg,HT_QlOJbDpg,0RViGi2Rne8,g9e1alirMhc,YZev1imoxX8,I4yKEK9o8gA,zQ83d_D2MGs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095,esCvu46qDaA,RSMFD,863.0,Entertainment,63.0,116929.0,4.84,182.0,222.0,8t4m5JM5r6Y,vOskZNJJII8,q3qdBVDZrmE,mLDLe024T_I,bhLbFGVduTA,tJGDW674hVo,TecLbWk8A3w,3Yb5YTEVlEo,e4yx1s-l6f8,yB62b8S3Ra4,2GHiAEeFQs0,pxcDZ9AohLM,G8OlUZEnyOA,M80K51DosFo
4096,kOZ8bRAO7YQ,RockstarGames,0.0,UNA,63.0,199361.0,4.79,1256.0,2410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4097,GSsVZcKHues,GTASite,1024.0,Entertainment,123.0,486752.0,4.86,1098.0,1764.0,TecLbWk8A3w,vOskZNJJII8,_X5nHLOiAVo,cNION6wGuso,5-reBn0gUkg,PHLmbD_dLh4,bhLbFGVduTA,zwp3HEwA1oM,XjR5e2W8n8E,ka8xZhlMvps,pP3oC2pf_EU,YDgd4V0ILNw,q3qdBVDZrmE,hKSk1bW9qzw
4098,foKRqJGcbPQ,scrambledeggsTV,1100.0,Entertainment,136.0,13115.0,4.45,33.0,37.0,LzI9VyIjpDI,_X5nHLOiAVo,5-reBn0gUkg,e8bEzU2Tx7c,e4yx1s-l6f8,hKSk1bW9qzw,WWdESkWpKvI,2GHiAEeFQs0,GSsVZcKHues,q3qdBVDZrmE,IbEW4N3DmI0,odXW0Uaj8aU,z84ko4Xzn40,Qjj_uMubxP0


In [0]:
# Define the column names
column_names = [
    "Video ID", 
    "Video Uploader", 
    "Interval between YouTube Establishment and Video Upload",
    "Video Category", 
    "Video Length", 
    "Number of Views", 
    "Rating on the Video", 
    "Number of Ratings", 
    "Number of Comments", 
    "Related Video IDs"
]


# Rename the first 9 columns using the provided names
for i in range(len(column_names)):
    df = df.withColumnRenamed(f"_c{i}", column_names[i])    
    

In [0]:
df.show()

+-----------+--------------------+-------------------------------------------------------+----------------+------------+---------------+-------------------+-----------------+------------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   Video ID|      Video Uploader|Interval between YouTube Establishment and Video Upload|  Video Category|Video Length|Number of Views|Rating on the Video|Number of Ratings|Number of Comments|Related Video IDs|       _c10|       _c11|       _c12|       _c13|       _c14|       _c15|       _c16|       _c17|       _c18|       _c19|       _c20|       _c21|       _c22|
+-----------+--------------------+-------------------------------------------------------+----------------+------------+---------------+-------------------+-----------------+------------------+-----------------+-----------+-----------+-----------+-----------+-----

In [0]:
from pyspark.sql.functions import concat_ws, col

# Columns to combine: from _c10 to _c22
columns_to_combine = [col(f'_c{i}') for i in range(10, 23)]

# Combine all relevant columns into a single column called "Related Video IDs"
df = df.withColumn("Related Video IDs", concat_ws(" ", *columns_to_combine))

# Drop the original _c10 to _c22 columns
columns_to_drop = [f'_c{i}' for i in range(10, 23)]
df = df.drop(*columns_to_drop)

In [0]:

# Show the result
df.show()

+-----------+--------------------+-------------------------------------------------------+----------------+------------+---------------+-------------------+-----------------+------------------+--------------------+
|   Video ID|      Video Uploader|Interval between YouTube Establishment and Video Upload|  Video Category|Video Length|Number of Views|Rating on the Video|Number of Ratings|Number of Comments|   Related Video IDs|
+-----------+--------------------+-------------------------------------------------------+----------------+------------+---------------+-------------------+-----------------+------------------+--------------------+
|QuRYeRnAuXM|EvilSquirrelPictures|                                                 1135.0|  Pets & Animals|         252|           1075|               4.96|               46|                86|nRcovJn9xHg 3TYqk...|
|3TYqkBJ9YRk|              hggh22|                                                 1135.0|          Comedy|         169|            228|    

In [0]:

from pyspark.sql.functions import col, regexp_replace

# Replace spaces with commas in the 'final_column' column
df = df.withColumn('Related Video IDs', regexp_replace(col('Related Video IDs'), ' ', ','))

In [0]:
df.toPandas()

,Video ID,Video Uploader,Interval between YouTube Establishment and Video Upload,Video Category,Video Length,Number of Views,Rating on the Video,Number of Ratings,Number of Comments,Related Video IDs
0,QuRYeRnAuXM,EvilSquirrelPictures,1135.0,Pets & Animals,252.0,1075.0,4.96,46.0,86.0,"nRcovJn9xHg,3TYqkBJ9YRk,rSJ8QZWBegU,0TZqX5MbXM..."
1,3TYqkBJ9YRk,hggh22,1135.0,Comedy,169.0,228.0,5.00,5.0,3.0,"gFa1YMEJFag,UEvVksP91kg,rSJ8QZWBegU,nRcovJn9xH..."
2,rSJ8QZWBegU,TimeGem,1135.0,Entertainment,95.0,356.0,4.31,13.0,1.0,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,nRcovJn9xH..."
3,nRcovJn9xHg,wooochacha,1135.0,Entertainment,118.0,1115.0,2.23,57.0,73.0,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,ZTopArY7Nb..."
4,UEvVksP91kg,johnx113,1135.0,Entertainment,83.0,281.0,2.67,9.0,16.0,"QuRYeRnAuXM,3TYqkBJ9YRk,0TZqX5MbXMA,rSJ8QZWBeg..."
...,...,...,...,...,...,...,...,...,...,...
4095,esCvu46qDaA,RSMFD,863.0,Entertainment,63.0,116929.0,4.84,182.0,222.0,"vOskZNJJII8,q3qdBVDZrmE,mLDLe024T_I,bhLbFGVduT..."
4096,kOZ8bRAO7YQ,RockstarGames,0.0,UNA,63.0,199361.0,4.79,1256.0,2410.0,
4097,GSsVZcKHues,GTASite,1024.0,Entertainment,123.0,486752.0,4.86,1098.0,1764.0,"vOskZNJJII8,_X5nHLOiAVo,cNION6wGuso,5-reBn0gUk..."
4098,foKRqJGcbPQ,scrambledeggsTV,1100.0,Entertainment,136.0,13115.0,4.45,33.0,37.0,"_X5nHLOiAVo,5-reBn0gUkg,e8bEzU2Tx7c,e4yx1s-l6f..."


In [0]:
df.count()

4100

In [0]:
# Drop Null Values 

In [0]:
df_cleaned=df.dropna(how="any")

In [0]:
df_cleaned.count()

4066

In [0]:
df_cleaned.printSchema()

root
 |-- Video ID: string (nullable = true)
 |-- Video Uploader: string (nullable = true)
 |-- Interval between YouTube Establishment and Video Upload: double (nullable = true)
 |-- Video Category: string (nullable = true)
 |-- Video Length: integer (nullable = true)
 |-- Number of Views: integer (nullable = true)
 |-- Rating on the Video: double (nullable = true)
 |-- Number of Ratings: integer (nullable = true)
 |-- Number of Comments: integer (nullable = true)
 |-- Related Video IDs: string (nullable = false)



In [0]:
data=df_cleaned

In [0]:
data.count()

4066

In [0]:
data.toPandas()

,Video ID,Video Uploader,Interval between YouTube Establishment and Video Upload,Video Category,Video Length,Number of Views,Rating on the Video,Number of Ratings,Number of Comments,Related Video IDs
0,QuRYeRnAuXM,EvilSquirrelPictures,1135.0,Pets & Animals,252,1075,4.96,46,86,"nRcovJn9xHg,3TYqkBJ9YRk,rSJ8QZWBegU,0TZqX5MbXM..."
1,3TYqkBJ9YRk,hggh22,1135.0,Comedy,169,228,5.00,5,3,"gFa1YMEJFag,UEvVksP91kg,rSJ8QZWBegU,nRcovJn9xH..."
2,rSJ8QZWBegU,TimeGem,1135.0,Entertainment,95,356,4.31,13,1,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,nRcovJn9xH..."
3,nRcovJn9xHg,wooochacha,1135.0,Entertainment,118,1115,2.23,57,73,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,ZTopArY7Nb..."
4,UEvVksP91kg,johnx113,1135.0,Entertainment,83,281,2.67,9,16,"QuRYeRnAuXM,3TYqkBJ9YRk,0TZqX5MbXMA,rSJ8QZWBeg..."
...,...,...,...,...,...,...,...,...,...,...
4061,esCvu46qDaA,RSMFD,863.0,Entertainment,63,116929,4.84,182,222,"vOskZNJJII8,q3qdBVDZrmE,mLDLe024T_I,bhLbFGVduT..."
4062,kOZ8bRAO7YQ,RockstarGames,0.0,UNA,63,199361,4.79,1256,2410,
4063,GSsVZcKHues,GTASite,1024.0,Entertainment,123,486752,4.86,1098,1764,"vOskZNJJII8,_X5nHLOiAVo,cNION6wGuso,5-reBn0gUk..."
4064,foKRqJGcbPQ,scrambledeggsTV,1100.0,Entertainment,136,13115,4.45,33,37,"_X5nHLOiAVo,5-reBn0gUkg,e8bEzU2Tx7c,e4yx1s-l6f..."


In [0]:
data.describe().toPandas()

,summary,Video ID,Video Uploader,Interval between YouTube Establishment and Video Upload,Video Category,Video Length,Number of Views,Rating on the Video,Number of Ratings,Number of Comments,Related Video IDs
0,count,4066,4066,4066,4066,4066,4066,4066,4066,4066,4066
1,mean,None,1.5376751983333333E9,956.7144613871127,None,232.90629611411705,194695.10944417116,3.7562592228234193,588.5696015740285,534.873831775701,None
2,stddev,None,3.76562331151571E9,214.1813221148061,None,219.475672454187,1488250.2854682289,1.6787591038040397,3767.893776063847,5032.528303622292,None
3,min,-08YZF87OBQ,000ILoveNaruto000,0.0,UNA,1,0,0.0,0,0,
4,max,zzlDKaXAIgE,zzzzzzzzap,1136.0,Travel & Events,5202,65341925,5.0,122129,259683,"zzlDKaXAIgE,e_Oun_GTKDo,4ira1eUOQGY,RWS45QqP1l..."


In [0]:
null_count=data.filter(data["Related Video IDs"].isNull()).count()
null_count

0

In [0]:
data.select('Related Video IDs').toPandas()

,Related Video IDs
0,"nRcovJn9xHg,3TYqkBJ9YRk,rSJ8QZWBegU,0TZqX5MbXM..."
1,"gFa1YMEJFag,UEvVksP91kg,rSJ8QZWBegU,nRcovJn9xH..."
2,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,nRcovJn9xH..."
3,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,ZTopArY7Nb..."
4,"QuRYeRnAuXM,3TYqkBJ9YRk,0TZqX5MbXMA,rSJ8QZWBeg..."
...,...
4061,"vOskZNJJII8,q3qdBVDZrmE,mLDLe024T_I,bhLbFGVduT..."
4062,
4063,"vOskZNJJII8,_X5nHLOiAVo,cNION6wGuso,5-reBn0gUk..."
4064,"_X5nHLOiAVo,5-reBn0gUkg,e8bEzU2Tx7c,e4yx1s-l6f..."


In [0]:
data[data['Related Video IDs']==''].count()

827

In [0]:
# Replace empty strings with null in 'Related Video IDs'
data_cleaned = data.withColumn(
    'Related Video IDs',
    when(col('Related Video IDs') == '', None).otherwise(col('Related Video IDs'))
)

In [0]:
data_cleaned[data_cleaned['Related Video IDs']==''].count()

0

In [0]:
data=data_cleaned

In [0]:
data.count()

4066

In [0]:
data.dropna()

DataFrame[Video ID: string, Video Uploader: string, Interval between YouTube Establishment and Video Upload: double, Video Category: string, Video Length: int, Number of Views: int, Rating on the Video: double, Number of Ratings: int, Number of Comments: int, Related Video IDs: string]

In [0]:
data.count()

4066

In [0]:
data = data.filter(col('Related Video IDs').isNotNull())

In [0]:
data.count()

3239

In [0]:
data.toPandas()

,Video ID,Video Uploader,Interval between YouTube Establishment and Video Upload,Video Category,Video Length,Number of Views,Rating on the Video,Number of Ratings,Number of Comments,Related Video IDs
0,QuRYeRnAuXM,EvilSquirrelPictures,1135.0,Pets & Animals,252,1075,4.96,46,86,"nRcovJn9xHg,3TYqkBJ9YRk,rSJ8QZWBegU,0TZqX5MbXM..."
1,3TYqkBJ9YRk,hggh22,1135.0,Comedy,169,228,5.00,5,3,"gFa1YMEJFag,UEvVksP91kg,rSJ8QZWBegU,nRcovJn9xH..."
2,rSJ8QZWBegU,TimeGem,1135.0,Entertainment,95,356,4.31,13,1,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,nRcovJn9xH..."
3,nRcovJn9xHg,wooochacha,1135.0,Entertainment,118,1115,2.23,57,73,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,ZTopArY7Nb..."
4,UEvVksP91kg,johnx113,1135.0,Entertainment,83,281,2.67,9,16,"QuRYeRnAuXM,3TYqkBJ9YRk,0TZqX5MbXMA,rSJ8QZWBeg..."
...,...,...,...,...,...,...,...,...,...,...
3234,S95602a7cFE,mattCTB,1045.0,Entertainment,87,39885,4.51,63,131,"bLAP45F4UCw,URIaUbRjIrU,x2TUkfeO-gM,Ydm5IedLYS..."
3235,esCvu46qDaA,RSMFD,863.0,Entertainment,63,116929,4.84,182,222,"vOskZNJJII8,q3qdBVDZrmE,mLDLe024T_I,bhLbFGVduT..."
3236,GSsVZcKHues,GTASite,1024.0,Entertainment,123,486752,4.86,1098,1764,"vOskZNJJII8,_X5nHLOiAVo,cNION6wGuso,5-reBn0gUk..."
3237,foKRqJGcbPQ,scrambledeggsTV,1100.0,Entertainment,136,13115,4.45,33,37,"_X5nHLOiAVo,5-reBn0gUkg,e8bEzU2Tx7c,e4yx1s-l6f..."


In [0]:
# Define the directory path to save the CSV files in DBFS
output_path = '/dbfs/Workspace/Users/d-user11@heu.ai/output.csv'

# Export the DataFrame to a CSV file
data.write.csv(output_path, header=True,mode='overwrite')

In [0]:
# List files in a directory
dbutils.fs.ls("/dbfs/Workspace/Users/d-user11@heu.ai/")

[FileInfo(path='dbfs:/dbfs/Workspace/Users/d-user11@heu.ai/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1724927607000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/d-user11@heu.ai/_committed_3854143736740846760', name='_committed_3854143736740846760', size=112, modificationTime=1724927153000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/d-user11@heu.ai/_committed_4262934571597625899', name='_committed_4262934571597625899', size=211, modificationTime=1724927607000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/d-user11@heu.ai/_started_3854143736740846760', name='_started_3854143736740846760', size=0, modificationTime=1724927152000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/d-user11@heu.ai/_started_4262934571597625899', name='_started_4262934571597625899', size=0, modificationTime=1724927606000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/d-user11@heu.ai/cleaneddata.csv/', name='cleaneddata.csv/', size=0, modificationTime=1724926558000),
 FileInfo(path='dbfs:/dbfs/Workspace/Users/d

In [0]:
# Read a CSV file from DBFS
dfo = spark.read.csv("/dbfs/Workspace/Users/d-user11@heu.ai/output.csv", header=True, inferSchema=True)
dfo.toPandas()



,Video ID,Video Uploader,Interval between YouTube Establishment and Video Upload,Video Category,Video Length,Number of Views,Rating on the Video,Number of Ratings,Number of Comments,Related Video IDs
0,QuRYeRnAuXM,EvilSquirrelPictures,1135.0,Pets & Animals,252,1075,4.96,46,86,"nRcovJn9xHg,3TYqkBJ9YRk,rSJ8QZWBegU,0TZqX5MbXM..."
1,3TYqkBJ9YRk,hggh22,1135.0,Comedy,169,228,5.00,5,3,"gFa1YMEJFag,UEvVksP91kg,rSJ8QZWBegU,nRcovJn9xH..."
2,rSJ8QZWBegU,TimeGem,1135.0,Entertainment,95,356,4.31,13,1,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,nRcovJn9xH..."
3,nRcovJn9xHg,wooochacha,1135.0,Entertainment,118,1115,2.23,57,73,"gFa1YMEJFag,UEvVksP91kg,3TYqkBJ9YRk,ZTopArY7Nb..."
4,UEvVksP91kg,johnx113,1135.0,Entertainment,83,281,2.67,9,16,"QuRYeRnAuXM,3TYqkBJ9YRk,0TZqX5MbXMA,rSJ8QZWBeg..."
...,...,...,...,...,...,...,...,...,...,...
3234,S95602a7cFE,mattCTB,1045.0,Entertainment,87,39885,4.51,63,131,"bLAP45F4UCw,URIaUbRjIrU,x2TUkfeO-gM,Ydm5IedLYS..."
3235,esCvu46qDaA,RSMFD,863.0,Entertainment,63,116929,4.84,182,222,"vOskZNJJII8,q3qdBVDZrmE,mLDLe024T_I,bhLbFGVduT..."
3236,GSsVZcKHues,GTASite,1024.0,Entertainment,123,486752,4.86,1098,1764,"vOskZNJJII8,_X5nHLOiAVo,cNION6wGuso,5-reBn0gUk..."
3237,foKRqJGcbPQ,scrambledeggsTV,1100.0,Entertainment,136,13115,4.45,33,37,"_X5nHLOiAVo,5-reBn0gUkg,e8bEzU2Tx7c,e4yx1s-l6f..."


In [0]:
# Analysis and Insights
# 1. Top Video Categories
# Objective: Find the top video categories with the maximum number of videos uploaded.

